In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score

from scipy import stats

In [2]:
train = pd.read_csv("./data/train.csv")
train_y = train[["SALE PRICE"]]
train_X = train
del train_X["SALE PRICE"]

test_X = pd.read_csv("./data/test.csv")
test_y = pd.read_csv("./data/test_groundtruth.csv")

print("train_X:",train_X.shape)
print("train_y:",train_y.shape)
print("test_X:",test_X.shape)
print("test_y:",test_y.shape)

train_X: (43064, 19)
train_y: (43064, 1)
test_X: (10767, 19)
test_y: (10767, 1)


In [3]:
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE DATE
0,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,R4,402 EAST 90TH STREET,5A,10128,1,0,1,-,-,1910,2,R4,2016-09-30 00:00:00
1,4,HOLLISWOOD,10 COOPS - ELEVATOR APARTMENTS,2,10538,70,D4,"87-50 204TH STREET, B42",,11423,0,0,0,-,-,1954,2,D4,2017-02-10 00:00:00
2,3,DOWNTOWN-FULTON MALL,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,R4,265 STATE STREET,910,11201,1,0,1,0,0,2014,2,R4,2017-01-25 00:00:00
3,4,FLUSHING-NORTH,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,R4,137-11 32 AVENUE,4W,11354,1,0,1,-,-,0,2,R4,2017-03-17 00:00:00
4,3,PARK SLOPE,09 COOPS - WALKUP APARTMENTS,2C,1067,29,C6,"862 PRESIDENT STREET, 1",,11215,0,0,0,0,0,1920,2,C6,2016-09-09 00:00:00


In [4]:
num_train_samples = len(train_X)

data_X = pd.concat([train_X, test_X])

# Data Pre Processing

In [5]:
# Let's delete some of the columns that we ** may not ** need
# 请注意 下面删除的特征很可能是有用的，合理的处理能够获得更为准确的预测模型，请探索所删除特征的使用
del data_X['ADDRESS']
del data_X['APARTMENT NUMBER']
del data_X['BUILDING CLASS AT PRESENT']
del data_X['BUILDING CLASS AT TIME OF SALE']
del data_X['NEIGHBORHOOD']
del data_X['SALE DATE']
del data_X['LAND SQUARE FEET']
del data_X['GROSS SQUARE FEET']

In [6]:
# Let's convert some of the columns to appropriate datatype

data_X['TAX CLASS AT TIME OF SALE'] = data_X['TAX CLASS AT TIME OF SALE'].astype('category')
data_X['TAX CLASS AT PRESENT'] = data_X['TAX CLASS AT PRESENT'].astype('category')
data_X['BOROUGH'] = data_X['BOROUGH'].astype('category')

In [7]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53831 entries, 0 to 10766
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   BOROUGH                    53831 non-null  category
 1   BUILDING CLASS CATEGORY    53831 non-null  object  
 2   TAX CLASS AT PRESENT       53831 non-null  category
 3   BLOCK                      53831 non-null  int64   
 4   LOT                        53831 non-null  int64   
 5   ZIP CODE                   53831 non-null  int64   
 6   RESIDENTIAL UNITS          53831 non-null  int64   
 7   COMMERCIAL UNITS           53831 non-null  int64   
 8   TOTAL UNITS                53831 non-null  int64   
 9   YEAR BUILT                 53831 non-null  int64   
 10  TAX CLASS AT TIME OF SALE  53831 non-null  category
dtypes: category(3), int64(7), object(1)
memory usage: 3.9+ MB


In [8]:
data_X.head()

,BOROUGH,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,YEAR BUILT,TAX CLASS AT TIME OF SALE
0,1,13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,10128,1,0,1,1910,2
1,4,10 COOPS - ELEVATOR APARTMENTS,2,10538,70,11423,0,0,0,1954,2
2,3,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,11201,1,0,1,2014,2
3,4,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,11354,1,0,1,0,2
4,3,09 COOPS - WALKUP APARTMENTS,2C,1067,29,11215,0,0,0,1920,2


In [9]:
#Select the variables to be one-hot encoded
one_hot_features = ['BOROUGH', 'BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','TAX CLASS AT TIME OF SALE']

In [10]:
# Convert categorical variables into dummy/indicator variables (i.e. one-hot encoding).
one_hot_encoded = pd.get_dummies(data_X[one_hot_features])
one_hot_encoded.info(verbose=True, memory_usage=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53831 entries, 0 to 10766
Data columns (total 59 columns):
 #   Column                                                                Non-Null Count  Dtype
---  ------                                                                --------------  -----
 0   BOROUGH_1                                                             53831 non-null  uint8
 1   BOROUGH_2                                                             53831 non-null  uint8
 2   BOROUGH_3                                                             53831 non-null  uint8
 3   BOROUGH_4                                                             53831 non-null  uint8
 4   BOROUGH_5                                                             53831 non-null  uint8
 5   BUILDING CLASS CATEGORY_01 ONE FAMILY DWELLINGS                       53831 non-null  uint8
 6   BUILDING CLASS CATEGORY_02 TWO FAMILY DWELLINGS                       53831 non-null  uint8
 7   BUILDING CLAS

In [11]:
data_X = data_X.drop(one_hot_features,axis=1)
data_X = pd.concat([data_X, one_hot_encoded] ,axis=1)

In [12]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53831 entries, 0 to 10766
Data columns (total 66 columns):
 #   Column                                                                Non-Null Count  Dtype
---  ------                                                                --------------  -----
 0   BLOCK                                                                 53831 non-null  int64
 1   LOT                                                                   53831 non-null  int64
 2   ZIP CODE                                                              53831 non-null  int64
 3   RESIDENTIAL UNITS                                                     53831 non-null  int64
 4   COMMERCIAL UNITS                                                      53831 non-null  int64
 5   TOTAL UNITS                                                           53831 non-null  int64
 6   YEAR BUILT                                                            53831 non-null  int64
 7   BOROUGH_1    

In [13]:
train_X = data_X[:num_train_samples].to_numpy()
test_X = data_X[num_train_samples:].to_numpy()

# Regression

In [16]:
rf_regr = RandomForestRegressor()
rf_regr.fit(train_X, train_y)
Y_pred_rf = rf_regr.predict(test_X)

# MAPE metric
mean_absolute_percentage_error(test_y,Y_pred_rf)

0.3672876637286009

In [15]:
pd.DataFrame({"pred":Y_pred_rf}).to_csv("学号_姓名.csv")